In [1]:
from requests import post
from app.analysis import Quote
query = 'query UnknownFile($limit_0:[FundLimitOptionInput!]!) {viewer {...Fe}} fragment Fe on Viewer {_screenFunds4s9d1a:screenFunds(filterCategory:["$$"],limit:$limit_0,first:100) {edges {node {symbol}}}}'
variables = {'limit_0':[{'type':'FUND_TOTAL_ASSETS','min':2000000000,'inclusive':True},{'type':'FUND_NET_EXPENSE_RATIO','max':.5,'inclusive':True}]}
r = post('https://lens.m1finance.com/graphql', json={'query': query.replace('$$', 'Equities","Domestic'), 'variables': variables}).json()
symbols = [n['node']['symbol'] for n in r['data']['viewer']['_screenFunds4s9d1a']['edges']]

In [2]:
quote = Quote(symbols, 120)
f'from {quote.start.date()} to {quote.end.date()} - {len(quote.data.columns)}'

'from 2019-02-28 to 2019-06-28 - 86'

In [3]:
stat = quote.statistics(21)
stat = stat[(stat['drawdown'] < 8) & (stat['21-shrp'] > 0.4)]
stat

,21-len,21-mean,21-std,21-shrp,drawdown
Symbols,,,,,
PGX,64,0.800158,0.431449,1.468289,1.237116
SPLV,64,2.097328,1.395772,1.383221,2.660443
USMV,64,2.173729,1.743270,1.151320,2.180427
PFF,64,0.801404,0.672079,0.944438,1.003023
MTUM,64,2.125345,2.093397,0.935646,2.859384
XMLV,64,1.318236,1.560093,0.738142,3.688597
VIG,64,1.761311,2.502991,0.637095,4.634193
VOT,64,1.674926,3.180653,0.474198,5.627906
VOOG,64,1.478941,2.899947,0.452517,6.391441


In [4]:
quote.setup_mask(stat.index)
quote.least_correlated_portfolio(21, 2)

['PGX'] -0.4208655143722255 0.6 0.24742325575833032 1.468288770130556
['PGX', 'SPLV'] -0.5549503159022314 0.4810487588032717 0.38975592885216226 1.4257550035576654


['PGX', 'SPLV']

In [5]:
quote.setup_mask(['PGX', 'SPLV'])
quote.find_optimal_ratio(21, 2)

/Users/chaoyi/.local/share/virtualenvs/portfolio_analysis-bdMx2S3t/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


({'PGX': 1.07, 'SPLV': -0.07}, 0.7141, 0.4234)